In [19]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as iplot
%matplotlib inline
plt.style.use('ggplot')
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
init_notebook_mode(connected=True)
cf.go_offline()

In [20]:
def createSignals(index_input, etf_input, rolling_window):
    index = yf.download(str(index_input), start="2015-01-01", end="2020-01-01")
    etf = yf.download(str(etf_input), start="2015-01-01", end="2020-01-01")
    
    index['sma'] = index['Close'].rolling(window=rolling_window).mean()
    
    index.drop(['Open','High','Low','Adj Close','Volume'], axis=1, inplace=True)
    etf.drop(['Open','High','Low','Adj Close','Volume'], axis=1, inplace=True)
    
    index['etf']=etf['Close']
    index.dropna(inplace=True)
    
    print(index.head())
    
    cond = input("Condition?") # index['sma'] < index['Close']
    index['cond'] = np.where(eval(cond), 1, -1)
    
    index['signal']=0
    rows=len(index)
    for i in range(1,rows):
        if index['cond'][i]==index['cond'][i-1]:
            index['signal'][i]=1
            i=i+1
    
    for j in range(0,rows):
        if index['signal'][j]==1:
            index['signal'][j]=0
            j=j+1
        else:
            index['signal'][j]=index['cond'][j]
            j=j+1

    return index

In [23]:
createSignals('^GSPC','SPY',10)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                  Close          sma         etf
Date                                            
2015-01-15  1992.670044  2026.946997  199.020004
2015-01-16  2019.420044  2023.069006  201.630005
2015-01-20  2022.550049  2023.266016  202.059998
2015-01-21  2032.119995  2026.217017  203.080002
2015-01-22  2063.149902  2029.942004  206.100006
Condition?index['sma'] < index['Close']


,Close,sma,etf,cond,signal
Date,,,,,
2015-01-15,1992.670044,2026.946997,199.020004,-1,-1
2015-01-16,2019.420044,2023.069006,201.630005,-1,0
2015-01-20,2022.550049,2023.266016,202.059998,-1,0
2015-01-21,2032.119995,2026.217017,203.080002,1,1
2015-01-22,2063.149902,2029.942004,206.100006,1,0
...,...,...,...,...,...
2019-12-24,3223.379883,3192.808984,321.230011,1,0
2019-12-26,3239.909912,3202.636987,322.940002,1,0
2019-12-27,3240.020020,3209.781982,322.859985,1,0


In [24]:
def backtestingPortfolio(index, initial_capital):
    initial_capital= float(initial_capital)

    index['shares']=0
    index.drop(['cond'],axis=1, inplace=True)
    index['balance']=float(0.0)
    index['balance'][0]=initial_capital
    
    for i in range(1,len(index)):
        if index.signal[i]==1:
            index['shares'][i]=0.1*index.balance[i-1]/index.etf[i]
            index['balance'][i]=0.9*index.balance[i-1]
            i=i+1
        elif index.signal[i]==-1:
            index['balance'][i]=index.balance[i-1]+index.shares[i-1]*index.etf[i]
            index['shares'][i]=0
            i=i+1
        else:
            index.balance[i]=index.balance[i-1]
            index.shares[i]=index.shares[i-1]

    index['Portfolio Value']=index.balance+index.etf*index.shares   

    return index.head(10), index['balance'].iplot(kind='line')

In [25]:
backtestingPortfolio(createSignals('^GSPC','SPY',10),100000)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                  Close          sma         etf
Date                                            
2015-01-15  1992.670044  2026.946997  199.020004
2015-01-16  2019.420044  2023.069006  201.630005
2015-01-20  2022.550049  2023.266016  202.059998
2015-01-21  2032.119995  2026.217017  203.080002
2015-01-22  2063.149902  2029.942004  206.100006
Condition?index['sma'] < index['Close']


(                  Close          sma         etf  signal  shares  \
 Date                                                               
 2015-01-15  1992.670044  2026.946997  199.020004      -1       0   
 2015-01-16  2019.420044  2023.069006  201.630005       0       0   
 2015-01-20  2022.550049  2023.266016  202.059998       0       0   
 2015-01-21  2032.119995  2026.217017  203.080002       1      49   
 2015-01-22  2063.149902  2029.942004  206.100006       0      49   
 2015-01-23  2051.820068  2028.910022  204.970001       0      49   
 2015-01-26  2057.090088  2030.138025  205.449997       0      49   
 2015-01-27  2029.550049  2030.267029  202.740005      -1       0   
 2015-01-28  2002.160034  2028.180029  200.139999       0       0   
 2015-01-29  2021.250000  2029.178027  201.990005       0       0   
 
                   balance  Portfolio Value  
 Date                                        
 2015-01-15  100000.000000    100000.000000  
 2015-01-16  100000.000000    10